### 1. functions

In [3]:
import pandas as pd
import numpy as np
import cplex
from cplex.exceptions import CplexSolverError
from cplex import SparsePair
from cplex.six.moves import zip
import time
import numba
from ortools.algorithms import pywrapknapsack_solver
from numba import jit

### 2. data

In [4]:
def data(ty,problem):
    
    f = open("gap_%s/%s.txt"%(ty,problem), 'r')
    data = f.readlines()
    f.close()

    records = []
    for line in data:
        record = [int(field) for field in line.strip().lstrip('[').rstrip(']').split()]
        records.append(record)

    size = records[0]
    agent = size[0]
    job = size[1]

    c = []
    a = []
    b = []
    for i in range(len(records)-1) :
        if len(c) < job*agent:
            c.extend(records[i+1])
        elif len(c) >= job*agent and len(a)< job*agent :
            a.extend(records[i+1])
        else :
            b.extend(records[i+1])

    c = np.array(c,dtype=int).reshape((agent,job))
    a = np.array(a,dtype=int).reshape((agent,job))
    b = np.array(b)
    
    return a,b,c,agent,job


### 3. Knapsacks

In [5]:
@jit
def sols(w,c):
    i = len(c)-1
    currentW =  len(c[0])-1

    marked = np.zeros(i+1)

    while i >= 0 and currentW >=0:
        if (i==0 and c[i][currentW] >0 )or c[i][currentW] != c[i-1][currentW]:
            marked[i] =1
            currentW = currentW-w[i]
        i = i-1
    return marked


@jit
def Knapsack1(v, w, W):

    n = len(v)

    c = np.zeros((n,W+1))

    for i in range(n):
        for j in range(W+1):
            if (w[i] > j):
                c[i][j] = c[i-1][j]
            else:
                c[i][j] = max(c[i-1][j],v[i] +c[i-1][j-w[i]])

    return c[n-1][W],sols(w,c)

@jit
def Knapsack2(val, wt, W): 

    n = len(val)
    K = np.zeros((n+1,W+1))
    for i in range(n + 1): 
        for w in range(W + 1): 
            if i == 0 or w == 0: 
                K[i][w] = 0
            elif wt[i - 1] <= w: 
                K[i][w] = max(val[i - 1] 
                + K[i - 1][w - wt[i - 1]], 
                        K[i - 1][w]) 
            else: 
                K[i][w] = K[i - 1][w] 
                
    res = K[n][W] 
    sol = np.zeros(n)
    w = W 
    print(W)
    for i in range(n, 0, -1): 
        if res <= 0: 
            break
        elif res == K[i - 1][w]: 
            continue
        else: 
            sol[i-1] = 1
            res = res - val[i - 1] 
            w = w - wt[i - 1] 
    return K[n][W],sol


try:
    xrange
except:
    xrange = range

# @jit(nopython=True)
@jit
def totalvalue(comb,W):
    totwt = totval = 0
    for wt, val in comb:
        totwt  += wt
        totval += val
    return totval if totwt <= W else (0, 0)
 
# @jit(nopython=True)
@jit
def Knapsack3(items, limit):
    table = [[0 for w in range(limit + 1)] for j in xrange(len(items) + 1)]
 
    for j in xrange(1, len(items) + 1):
        wt, val = items[j-1]
        for w in xrange(1, limit + 1):
            if wt > w:
                table[j][w] = table[j-1][w]
            else:
                table[j][w] = max(table[j-1][w],
                                  table[j-1][w-wt] + val)
 
    result = []
    sol = np.zeros(len(items))
    w = limit
    for j in range(len(items), 0, -1):
        was_added = table[j][w] != table[j-1][w]
 
        if was_added:
            wt, val = items[j-1]
            result.append(items[j-1])
            w -= wt
            sol[j-1] = 1.

 
    return result,sol
 

def Knapsack(val, wt, W): 
    
    wt = [wt]
    W = [W]
    solver = pywrapknapsack_solver.KnapsackSolver(
      pywrapknapsack_solver.KnapsackSolver.
      KNAPSACK_DYNAMIC_PROGRAMMING_SOLVER,
      'test')
    
    solver.Init(val, wt, W)
    computed_value = solver.Solve()
    
    sol = np.zeros(len(val))
    for x in range(len(val)):
        if solver.BestSolutionContains(x):
            sol[x] = 1.

    return computed_value, sol



import numpy
import ctypes
knapsack = ctypes.CDLL('knapsack.so')
knapsack.knapsack_bnb.restype = ctypes.c_double


def KnapsackBnB(profits, weights, b):
    
    n = len(profits)
    
    zero_list = [0] * n
    x = (ctypes.c_int * n)(*zero_list)
    one_list = [1] * n
    m = (ctypes.c_int * n)(*one_list)

    p = (ctypes.c_double * n)(*profits)
    w = (ctypes.c_int * n)(*weights)
    
    z = knapsack.knapsack_bnb(n, p, w, m, x, b)

    return z, list(x)



### 4. Kelly

In [31]:


def DP_General_CG(a,b,c,agent,job):
    
    iterations = []
    
    K = range(1)
    var = list(range(agent))

    M = cplex.Cplex()
    M.parameters.lpmethod.set(1)

    x_i_k = lambda i,k: 'x_%d_%d' % (i,k)
    x = [x_i_k(i,k) for i in range(1) for k in K]

    dummy = float(sum(np.sum(c,axis=1)))

    M.variables.add(
        lb = [0] * len(x),
        ub = [1] * len(x),
        names = x,
        obj =  [dummy],
        types = ['C'] * len(x)
    )



    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind =[x_i_k(i,k) for i in range(1) for k in K], 
                val = [1.0]
                )
        for j in range(job)
        ],
        senses=["G" for j in range(job)],
        rhs=[1.0 for j in range(job)] ,
        names=['assignment_%d' % (j) for j in range(job)])


    
    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind =[x_i_k(0,k) for k in K], 
                val = [0] * len(K)
                )
        for i in range(agent)
        ],
        senses=["L" for i in range(agent)],
        rhs=[1.0 for i in range(agent)] )

    
    for ag in range(agent):   

        w = list(a[ag])
        v = list(np.array(c[ag]))
        W = int(b[ag])

        S_obj, sol = KnapsackBnB(v, w, W)

        newsub = sol
        idx = M.variables.get_num()
        M.variables.add(obj=[float(np.array(sol).T @ c[ag])])
        M.linear_constraints.set_coefficients(list(zip(list(range(job)),
                                                         [idx] * job,
                                                         newsub)))

        M.linear_constraints.set_coefficients(job+ag, idx, 1.0)
        var.append(idx)
 
    M.objective.set_sense(M.objective.sense.minimize)    
    M.set_log_stream(None)
    M.set_error_stream(None)
    M.set_warning_stream(None)
    M.set_results_stream(None)

    start = time.time()
    mt = 0
    st = 0
    ite = 0
    solutions = []
    criteria = [True]*agent

    while any(criteria):

        ite+=1

        M.set_problem_type(M.problem_type.LP)
        ct = time.time()

        M.solve()
        solutions.append(float(M.solution.get_objective_value()))
        iterations.append(float(cplex._internal._procedural.getitcnt(M._env._e, M._lp)))
        mt += time.time() - ct

        pi = list(M.solution.get_dual_values())[:job]
        dual = list(M.solution.get_dual_values())
        
        for ag in range(agent):   
            
            w = list(a[ag])
            v = list(np.array(pi) -np.array(c[ag]))
            W = int(b[ag])

            pt = time.time()

            S_obj, sol = KnapsackBnB(v, w, W)

            st += time.time() - pt

            if S_obj - 0.00001 > -dual[job+ag]:

                criteria[ag] = True
                newsub = sol
                idx = M.variables.get_num()
                M.variables.add(obj=[float(np.array(sol).T @ c[ag])])
                M.linear_constraints.set_coefficients(list(zip(list(range(job)),
                                                                 [idx] * job,
                                                                 newsub)))

                M.linear_constraints.set_coefficients(job+ag, idx, 1.0)
                var.append(idx)
            else :
                criteria[ag] = False

    M.set_problem_type(M.problem_type.LP)
    ct = time.time()
    M.solve()
    solutions.append(float(M.solution.get_objective_value()))
    iterations.append(float(cplex._internal._procedural.getitcnt(M._env._e, M._lp)))
    mt += time.time()- ct
    tt = time.time()- start
    
    return ite,M,mt,st,tt,solutions,iterations


### 5. Stab.

In [11]:
#### This code should be reconsidered.
def Stabilization(a,b,c,agent,job):
    
    K = range(1)
    var = list(range(agent))
    eps = 0.1

    M = cplex.Cplex()

    x_i_k = lambda i,k: 'x_%d_%d' % (i,k)
    x = [x_i_k(i,k) for i in range(1) for k in K]

    dummy = float(sum(np.sum(c,axis=1)))

    M.variables.add(
        lb = [0] * len(x),
        ub = [1] * len(x),
        names = x,
        obj =  [dummy],
        types = ['C'] * len(x)
    )


    gp_j = lambda j: 'gp_%d' % (j)

    gp = [gp_j(j) for j in range(job)]

    M.variables.add(
        lb = [0] * len(gp),
        ub = [eps] * len(gp),
        names = gp,
        obj =  [100] * len(gp),
        types = ['C'] * len(gp)
    )


    gm_j = lambda j: 'gm_%d' % (j)

    gm = [gm_j(j) for j in range(job)]

    M.variables.add(
        lb = [0] * len(gm),
        ub = [eps] * len(gm),
        names = gm,
        obj =  [-100] * len(gm),
        types = ['C'] * len(gm)
    )

    yp_i = lambda i: 'yp_%d' % (i)

    yp = [yp_i(i) for i in range(agent)]

    M.variables.add(
        lb = [0] * len(yp),
        ub = [eps] * len(yp),
        names = yp,
        obj =  [100] * len(yp),
        types = ['C'] * len(yp)
    )

    ym_i = lambda i: 'ym_%d' % (i)

    ym = [ym_i(i) for i in range(agent)]

    M.variables.add(
        lb = [0] * len(ym),
        ub = [eps] * len(ym),
        names = ym,
        obj =  [-100] * len(ym),
        types = ['C'] * len(ym)
    )

    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind = x + [gp_j(j)] + [gm_j(j)] , 
                val = [1.0] + [1.0, -1.0]
                )
        for j in range(job)
        ],
        senses=["G" for j in range(job)],
        rhs=[1.0 for j in range(job)] ,
        names=['assignment_%d' % (j) for j in range(job)])


    M.linear_constraints.add(
        lin_expr= [
            cplex.SparsePair(
                ind = x + [yp_i(i)] + [ym_i(i)] , 
                val = [0] * len(K) + [1.0] + [-1.0]
                )
        for i in range(agent)
        ],
        senses=["L" for i in range(agent)],
        rhs=[1.0 for i in range(agent)] )


    M.objective.set_sense(M.objective.sense.minimize)    
    
    M.set_log_stream(None)
    M.set_error_stream(None)
    M.set_warning_stream(None)
    M.set_results_stream(None)


    start = time.time()
    mt = 0
    st = 0
    ite = 0

    criteria = [True]*agent
    t_total = time.time()

    while any(criteria):

        ite+=1

        M.set_problem_type(M.problem_type.LP)
#         M.write('m.lp')
        ct = time.time()
        M.solve()
        iterations.append(float(cplex._internal._procedural.getitcnt(M._env._e, M._lp)))
        mt += time.time() - ct
        
        dual = list(M.solution.get_dual_values())
        pi = dual[:job]
        phi = dual[job:]

        
        for ag in range(agent):   
            w = list(a[ag])
            v = list(np.array(pi) -np.array(c[ag]))
            W = int(b[ag])

            items = list(zip(w,v))

            pt = time.time()

            bagged , sol = Knapsack3(items, W)


            S_obj = totalvalue(bagged,W)
            st += time.time() - pt
        

            if (S_obj-0.000001 > -dual[job+ag]) or eps != 0:
                criteria[ag] = True

                M.objective.set_linear(zip(gp+gm+yp+ym , pi+list(-np.array(pi))+phi+list(-np.array(phi))))

                newsub = sol
                idx = M.variables.get_num()
                M.variables.add(obj=[sol.T @ c[ag]])
                M.linear_constraints.set_coefficients(list(zip(list(range(job)),
                                                                 [idx] * job,
                                                                 newsub)))

                M.linear_constraints.set_coefficients(job+ag, idx, 1.0)
                var.append(idx)

                if ite % 100 == 0 :
                    eps *= 0.1
                    if ite == 600 :
                        eps =0

                    for dv in gm+gp+ym+yp:
                        M.variables.set_upper_bounds(dv , eps )



            else :
                criteria[ag] = False



    M.set_problem_type(M.problem_type.LP)
    ct = time.time()
    M.solve()
    iterations.append(float(cplex._internal._procedural.getitcnt(M._env._e, M._lp)))
    solutions.append(float(M.solution.get_objective_value()))
#     print(M.solution.get_objective_value())
    mt += time.time()- ct
    tt = time.time()- start
    
    return ite,M,mt,st,tt



### 6. Exe.

In [35]:

K_results = {}
S_results = {}
# problems = ['d10100']
problems = ['d05100','d10100','d10200','d20100','d20200','e05100','e10100','e10200','e20100','e20200']

for problem in problems : 
    
    ty = problem[0]
    
    a,b,c,agent,job = data(ty, problem)
    
    ite,M,mt,st,tt,solutions,iterations = DP_General_CG(a,b,c,agent,job)
    per = mt/(st+mt)
    K_results[problem] = ['Kelly',ite,mt,st,tt,per,solutions,np.average(np.array(iterations))]
    
    
#     ite_s,M_s,mt,st,tt = Stabilization(a,b,c,agent,job)
#     per = mt/(st+mt)
#     S_results[problem] = ['Stab.',ite_s,mt,st,tt,per]
    
    print(problem, ' done!')

d05100  done!
d10100  done!
d10200  done!
d20100  done!
d20200  done!
e05100  done!
e10100  done!
e10200  done!
e20100  done!
e20200  done!


### 7. Results

In [36]:
re = pd.DataFrame(K_results)
re = re.transpose()
name = ['method','iteration','M','S','total','M_per','sol','ite']
re.columns = name

re1 = re
re1.drop(['M','S','sol'], axis=1, inplace=True)
re

,method,iteration,total,M_per,ite
d05100,Kelly,746,5.50198,0.88373,169.772
d10100,Kelly,167,1.13157,0.730506,105.256
d10200,Kelly,831,34.4322,0.922773,726.298
d20100,Kelly,75,0.850106,0.525675,121.842
d20200,Kelly,235,7.53839,0.812245,357.038
e05100,Kelly,597,3.3999,0.870159,126.341
e10100,Kelly,193,1.20748,0.736731,111.727
e10200,Kelly,987,44.5684,0.942454,848.692
e20100,Kelly,93,0.799529,0.586679,135.128
e20200,Kelly,244,7.93567,0.847073,495.294


In [46]:
re = pd.DataFrame(S_results)
re = re.transpose()
name = ['method','iteration','M','S','total','M_per']
re.columns = name

re2 =re
re2.drop(['M','S'], axis=1, inplace=True)
re

,method,iteration,total,M_per
d05100,Stab.,601,32.1572,0.127018
d10100,Stab.,601,45.3016,0.0987634
d10200,Stab.,636,146.207,0.280186
d20100,Stab.,601,64.3773,0.103514
d20200,Stab.,601,156.282,0.0966122
e05100,Stab.,601,19.2966,0.252199
e10100,Stab.,601,30.4032,0.178809
e10200,Stab.,938,141.514,0.500034
e20100,Stab.,601,55.798,0.1486
e20200,Stab.,601,116.472,0.218146


In [47]:
re_new = pd.concat([re1, re2], axis=1) 
re_new.to_csv('ori.csv')

re_new

,method,iteration,total,M_per,method,iteration,total,M_per
d05100,Kelly,827,36.467,0.160998,Stab.,601,32.1572,0.127018
d10100,Kelly,253,11.3628,0.0621515,Stab.,601,45.3016,0.0987634
d10200,Kelly,931,182.835,0.28898,Stab.,636,146.207,0.280186
d20100,Kelly,165,10.2006,0.0419115,Stab.,601,64.3773,0.103514
d20200,Kelly,401,74.8917,0.128235,Stab.,601,156.282,0.0966122
e05100,Kelly,717,14.2291,0.232564,Stab.,601,19.2966,0.252199
e10100,Kelly,274,7.17376,0.104853,Stab.,601,30.4032,0.178809
e10200,Kelly,1055,113.532,0.491966,Stab.,938,141.514,0.500034
e20100,Kelly,165,7.32217,0.0509024,Stab.,601,55.798,0.1486
e20200,Kelly,400,44.149,0.207435,Stab.,601,116.472,0.218146


In [37]:
import pickle

with open('result_K_init.txt', 'wb') as f:
    pickle.dump(K_results, f)
    pickle.dump(S_results, f)
    